In [1]:
from SimuSamp.new_funcs.load.SpatDat import SpatDat
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from SimuSamp.new_funcs.compute.n_core_sampler import sample_n_cores
import pingouin as pg

In [2]:
parent_filepath = "C:/Users/Michael/Desktop/block_data"
sampleids = ["01_F", "01_G", "02_G", "02_H", "03_B", "03_H", "04_I", "04_N", "05_G", "05_I", "06_H", "07_K", "08_J", "09_E", "10_B", "11_E", "11_F", "12_M", "13_G", "14_I", "15_G", "16_B", "17_I", "18_E", "19_F", "21_I", "22_N", "23_B", "24_I", "25_M", "26_I", "27_H", "28_FSC", "29_L", "30_D"]
microns_per_pixel=0.22715
mm2_per_pixels2 = (microns_per_pixel / 1000) ** 2

In [3]:
# single iteration
simdata_list = []
for sampleid in sampleids:
    
    spatdat = SpatDat(sampleid=sampleid, parent_filepath=parent_filepath)
    
    cells = spatdat.subset_cells("extended_partition")
    tum = spatdat.subset_annotation("tumour")
    ext_partition = spatdat.subset_annotation("extended_partition")
    im = spatdat.subset_annotation("IM")
    outer_im = spatdat.subset_annotation("outer_IM")

    for region in ["tumour"]:
        for r in [0.5]:
            for n in [1]:
                for i in range(1):
                    simdata = sample_n_cores(sampleid, cells, tum, region=region, secondary_anno=im, outer_im_anno=outer_im, core_radius=r, n_cores=n, extended_partition=ext_partition, n_neighbours=True)

                    simdata_list.append(simdata)

sim_data = pd.DataFrame(simdata_list)

In [4]:
sim_data

,Density_n_mean,Den_stdev,Den_sterr,Region,Sampleid,Radius,n_cores,Cores_actually_sampled,Counts_n_mean,Areas_n_mean,Nearest_neighbour_mean,Hopkins_mean,Density_top_core,Density_bottom_core
0,974.319386,0.0,0.0,tumour,01_F,500.0,1,1,764.0,0.784137,14.520094,0.835792,974.319386,974.319386
1,180.947858,0.0,0.0,tumour,01_G,500.0,1,1,114.0,0.630016,31.407456,0.823629,180.947858,180.947858
2,595.655505,0.0,0.0,tumour,02_G,500.0,1,1,297.0,0.498610,16.857298,0.828138,595.655505,595.655505
3,451.451653,0.0,0.0,tumour,02_H,500.0,1,1,354.0,0.784137,19.744139,0.754918,451.451653,451.451653
4,8.472587,0.0,0.0,tumour,03_B,500.0,1,1,6.0,0.708166,53.901299,0.484729,8.472587,8.472587
5,58.137222,0.0,0.0,tumour,03_H,500.0,1,1,39.0,0.670827,44.935258,0.632263,58.137222,58.137222
6,140.281587,0.0,0.0,tumour,04_I,500.0,1,1,110.0,0.784137,28.262390,0.894650,140.281587,140.281587
7,550.379373,0.0,0.0,tumour,04_N,500.0,1,1,323.0,0.586868,16.824426,0.844437,550.379373,550.379373
8,391.561036,0.0,0.0,tumour,05_G,500.0,1,1,282.0,0.720194,17.348955,0.766287,391.561036,391.561036
9,876.122275,0.0,0.0,tumour,05_I,500.0,1,1,687.0,0.784137,16.187248,0.814724,876.122275,876.122275


#### Intraclass Correlation Coefficient

In [5]:
samples = []
tum_dens = []
im_dens = []
for sampleid in sampleids:
    spatdat = SpatDat(sampleid, parent_filepath)
    tum_den = (len(spatdat.subset_cells("tumour")) / (spatdat.subset_annotation("tumour").area * mm2_per_pixels2))
    im_den = (len(spatdat.subset_cells("IM")) / (spatdat.subset_annotation("IM").area * mm2_per_pixels2))
    samples.append(sampleid)
    tum_dens.append(tum_den)
    im_dens.append(im_den)

den = pd.DataFrame({"Sampleid": samples, "tum_den": tum_dens, "im_den": im_dens})

In [6]:
den

,Sampleid,tum_den,im_den
0,01_F,804.022884,815.782184
1,01_G,225.027071,212.653527
2,02_G,1049.946522,941.210820
3,02_H,431.779689,416.352022
4,03_B,33.618035,162.369037
5,03_H,91.502656,321.243087
6,04_I,346.394474,480.472781
7,04_N,363.616220,546.144162
8,05_G,379.291157,802.300236
9,05_I,423.454086,847.909409


In [24]:
tma = sim_data[["Sampleid", "Region", "Density_n_mean"]]

tma_im = tma[tma["Region"] == "IM"].reset_index(drop=True)
tma_tum = tma[tma["Region"] == "tumour"].reset_index(drop=True)

# den = den.rename(columns={"sampleid": "Sampleid"})

icc_table_im = tma_im.merge(den, on="Sampleid", how="left")
icc_table_tum = tma_tum.merge(den, on="Sampleid", how="left")

icc_table_im = icc_table_im[~icc_table_im["Density_n_mean"].isna()]
icc_table_tum = icc_table_tum[~icc_table_tum["Density_n_mean"].isna()]

In [25]:
icc_table_tum_long = icc_table_tum.melt(
    id_vars="Sampleid",
    value_vars=["Density_n_mean", "tum_den"],
    var_name="raters",
    value_name="ratings"
)

icc_table_im_long = icc_table_im.melt(
    id_vars="Sampleid",
    value_vars=["Density_n_mean", "tum_den"],
    var_name="raters",
    value_name="ratings"
)

In [26]:
icc_table_tum_long

,Sampleid,raters,ratings
0,01_F,Density_n_mean,587.968742
1,01_G,Density_n_mean,375.402162
2,02_G,Density_n_mean,1202.297157
3,02_H,Density_n_mean,230.826975
4,03_B,Density_n_mean,115.207468
...,...,...,...
65,26_I,tum_den,265.230197
66,27_H,tum_den,706.907380
67,28_FSC,tum_den,127.185324
68,29_L,tum_den,116.881187


In [29]:
icc_res_tum = pg.intraclass_corr(icc_table_tum_long, targets="Sampleid", ratings="ratings", raters="raters")
icc_res_tum

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.744210,6.818916,34,35,7.095328e-08,"[0.55, 0.86]"
1,ICC2,Single random raters,0.744480,6.875724,34,34,8.822848e-08,"[0.55, 0.86]"
2,ICC3,Single fixed raters,0.746055,6.875724,34,34,8.822848e-08,"[0.55, 0.86]"
3,ICC1k,Average raters absolute,0.853349,6.818916,34,35,7.095328e-08,"[0.71, 0.93]"
4,ICC2k,Average random raters,0.853527,6.875724,34,34,8.822848e-08,"[0.71, 0.93]"
5,ICC3k,Average fixed raters,0.854561,6.875724,34,34,8.822848e-08,"[0.71, 0.93]"


In [30]:
icc_res_im = pg.intraclass_corr(icc_table_im_long, targets="Sampleid", ratings="ratings", raters="raters")
icc_res_im

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.494782,2.958684,30,31,0.001791,"[0.18, 0.72]"
1,ICC2,Single random raters,0.517955,3.654076,30,30,0.000328,"[0.19, 0.74]"
2,ICC3,Single fixed raters,0.570269,3.654076,30,30,0.000328,"[0.28, 0.77]"
3,ICC1k,Average raters absolute,0.662012,2.958684,30,31,0.001791,"[0.3, 0.84]"
4,ICC2k,Average random raters,0.682438,3.654076,30,30,0.000328,"[0.32, 0.85]"
5,ICC3k,Average fixed raters,0.726333,3.654076,30,30,0.000328,"[0.43, 0.87]"


### Single Sample Test

In [3]:
spatdat = SpatDat(sampleid="13_G", parent_filepath=parent_filepath)

In [4]:
cells = spatdat.subset_cells("extended_partition")
tum = spatdat.subset_annotation("tumour")
ext_partition = spatdat.subset_annotation("extended_partition")
im = spatdat.subset_annotation("IM")
outer_im = spatdat.subset_annotation("outer_IM")

In [6]:
sample_n_cores("13_G", cells, tum, region="IM", secondary_anno=im, outer_im_anno=outer_im, core_radius=0.5, n_cores=10, extended_partition=ext_partition)

{'Density_n_mean': 212.76433846359083,
 'Den_stdev': 119.06050144587343,
 'Den_sterr': 39.686833815291145,
 'Region': 'IM',
 'Sampleid': '13_G',
 'Radius': 2201.188641866608,
 'n_cores': 10,
 'Cores_actually_sampled': 9,
 'Counts_n_mean': 144.77777777777777,
 'Areas_n_mean': 0.6989862936045439,
 'Density_top_core': 405.6898316475258,
 'Density_bottom_core': 24.23045593877351}